In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
df = pd.read_csv('/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv')
df.head()

In [ ]:
df.Date = pd.to_datetime(df.Date)
df.set_index('Date',inplace=True)

In [ ]:
df.groupby('RainTomorrow').mean()

In [ ]:
df.Evaporation.fillna(value=df.Evaporation.mean(),inplace=True)

In [ ]:
sns.boxplot(x=df.WindGustDir,y=df.WindGustSpeed) # implying direction isn't as important, hence dropped

In [ ]:
df.drop(columns=['WindGustDir','WindDir9am','WindDir3pm','RISK_MM'],inplace=True)

In [ ]:
sns.boxplot(x=df.RainTomorrow,y=df.Sunshine)

We see a stark difference in the hours of sunlight on a day when it rains next day vs when it doesn't, hence we fill the NA's with the mean values depending on whether it rained or not next day

I couldn't find a means to fill the na's directly, hence had to take a detour.

In [ ]:
df.Sunshine.fillna(value='Missing', inplace=True)

In [ ]:
def sunshine_fillna(row):
    if row[0] == 'Missing':
        if row[1] == 'Yes':
            return 4.47
        else:
            return 8.55
    else:
        return row[0]

In [ ]:
df['Sunshine'] = df[['Sunshine','RainTomorrow']].apply(sunshine_fillna,axis=1)

In [ ]:
sns.boxplot(x=df.RainTomorrow,y=df.WindGustSpeed) # similarly for Wind  Gust Speed

In [ ]:
df.WindGustSpeed.fillna(value='Missing', inplace=True)

In [ ]:
def wgs_fillna(row):
    if row[0] == 'Missing':
        if row[1] == 'Yes':
            return 45.95
        else:
            return 38.29
    else:
        return row[0]

In [ ]:
df['WindGustSpeed'] = df[['WindGustSpeed','RainTomorrow']].apply(wgs_fillna,axis=1)

In [ ]:
df.Humidity9am.fillna('Missing',inplace=True)
df.Humidity3pm.fillna('Missing',inplace=True)

In [ ]:
def hum9am_fillna(row):
    if row[0] == 'Missing':
        if row[1] == 'Yes':
            return 77.98
        else:
            return 66.22
    else:
        return row[0]

In [ ]:
def hum3pm_fillna(row):
    if row[0] == 'Missing':
        if row[1] == 'Yes':
            return 68.80
        else:
            return 46.51
    else:
        return row[0]

In [ ]:
df['Humidity9am'] = df[['Humidity9am','RainTomorrow']].apply(hum9am_fillna,axis=1)

In [ ]:
df['Humidity3pm'] = df[['Humidity3pm','RainTomorrow']].apply(hum3pm_fillna,axis=1)

In [ ]:
sns.boxplot(x=df.RainTomorrow,y=df.Pressure9am) # doesn't seem significant, hence dropped

In [ ]:
df.drop(columns=['Pressure9am','Pressure3pm'],inplace=True)

In [ ]:
df.Cloud9am.fillna('Missing',inplace=True)
df.Cloud3pm.fillna('Missing',inplace=True)

In [ ]:
def cloud9am_fillna(row):
    if row[0] == 'Missing':
        if row[1] == 'Yes':
            return 6.1
        else:
            return 3.93
    else:
        return row[0]

In [ ]:
def cloud3pm_fillna(row):
    if row[0] == 'Missing':
        if row[1] == 'Yes':
            return 6.36
        else:
            return 3.92
    else:
        return row[0]

In [ ]:
df['Cloud9am'] = df[['Cloud9am','RainTomorrow']].apply(cloud9am_fillna,axis=1)

In [ ]:
df['Cloud3pm'] = df[['Cloud3pm','RainTomorrow']].apply(cloud3pm_fillna,axis=1)

In [ ]:
df.Temp9am.fillna(df.Temp9am.mean(),inplace=True)
df.Temp3pm.fillna(df.Temp3pm.mean(),inplace=True)

In [ ]:
sns.boxplot(x=df.RainTomorrow,y=df.MinTemp) # indistinguishable, hence dropped.

In [ ]:
df.drop(columns=['MinTemp','MaxTemp'],inplace=True)

In [ ]:
sns.boxplot(x=df.RainToday,y=df.WindSpeed9am) # indistinguishable, hence drop both

In [ ]:
df.drop(columns=['WindSpeed9am','WindSpeed3pm'],inplace=True)

In [ ]:
df.drop(columns=['RainToday'],inplace=True)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.info()

**NOW WE'RE ALL SET TO DO SOME PREDICTIONS!**

**BUT BEFORE THAT WE NEED ALL FEATURES TO BE STANDARDIZED, SINCE THEY ARE IN DIFFERENT UNITS**

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled = scaler.fit_transform(df.drop(columns=['Location','RainTomorrow']))
X = pd.DataFrame(scaled,columns=[['Rainfall', 'Evaporation', 'Sunshine', 'WindGustSpeed',
       'Humidity9am', 'Humidity3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm']])

In [ ]:
X.head()

In [ ]:
y = df.RainTomorrow

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
model.score(X_test,y_test)

***A 93.87 % ACCURACY ISN'T THAT BAD!***